# Livedoor News Corpus のカテゴリ分類


## Livedoor News Corpus の準備

### Livedoor News Corpusとは

[Livedoor News Corpus](https://www.rondhuit.com/download.html#news%20corpus)は、ライブドアニュースの記事を利用して作成されたテキストデータセットです。

このデータには，以下の情報が含まれています．
- 記事のURL
- 記事の日付
- 記事のタイトル
- 記事の本文
- 記事のカテゴリ

記事のカテゴリは，"トピックニュース"，"Sports Watch"，"ITライフハック"，"家電チャンネル"，"MOVIE ENTER"，"独女通信"，"エスマックス，"livedoor HOMME"，"Peachy"の9つからなります．

記事は全部で7,367件あります．

元々はテキストファイルとして公開されていますが，Hugging Face Datasetsに登録されているため，今回はこれを利用します．

### データの読み込みと確認

Hugging Face Datasetsを利用してデータを読み込みます:

In [1]:
from datasets import load_dataset

dataset = load_dataset("shunk031/livedoor-news-corpus")

データは，`train`，`validation`，`test`の3つに分かれています．

In [2]:
# datasetのキーを表示する
print(dataset.keys())

dict_keys(['train', 'validation', 'test'])


データの中身を確認してみましょう:

In [3]:
from pprint import pprint
pprint(dataset["train"][0])

{'category': 3,
 'content': 'NHKの情報番組「お元気ですか日本列島」内の「ことばおじさんの気になることば」は、毎回、言葉の疑問に迫っていくコーナーだが、24日に放送された「日本に浸透している韓国語」の内容が、ネットユーザーの間で注目を集めている。  '
            '放送によると、いま日本の若者の間では、携帯メールでハングルの絵文字を使うのがブームだと伝えている。「ハングルはかわいくてデザインにしやすい」と感じる人が増えているそうだ。また、若者へのインタビューでも「韓国語のほうが素直に言える。日本語だと恥ずかしい」「日本語では謝りにくいが『ミアネヨ、オンマ』（ごめんね、ママ）だと言いやすい」と答えており、実際にハングルを使ったメールも紹介された。  '
            'これに対してネットユーザーは「そんな話聞いたことない」「こんなメール来たら縁を切るわ」など、番組が特集した“ブーム”の存在に疑問を呈する声が続出。また、「フジかと思ったらNHKかよ」「受信料払いたくない」「今度はNHKデモか?ww」など、NHKが韓国寄りの番組を放送していたことに批判的なネットユーザーの声も目立った。  '
            '【関連情報】 ・「ことばおじさんの気になることば」  ・今、日本語にさりげなく韓国語を混ぜるのが大ブーム',
 'date': '2011-11-25T19:48:00+0900',
 'title': 'NHKの″韓流寄り″番組に批判の声',
 'url': 'http://news.livedoor.com/article/detail/6062449/'}


この結果から，データは
- 'url' : 記事のURL
- 'date' : 記事の日付
- 'title' : 記事のタイトル
- 'content' : 記事の本文
- 'category' : 記事のカテゴリ
で与えられることがわかりました．

`train`, `valudation`, `test` それぞれでカテゴリの分布を確認してみます．

In [4]:
# trainのcategoryの分布を計算する
from collections import Counter

for set_name in dataset.keys():
    category_counter = Counter([data["category"] for data in dataset[set_name]])
    print(f"{set_name}: {category_counter}")


train: Counter({6: 900, 0: 870, 1: 870, 8: 870, 5: 842, 2: 772, 3: 770})
validation: Counter({4: 511, 7: 134, 2: 92})
test: Counter({7: 736})


このラベルのバランスでは適切な学習ができないため，改めてバランスをとりなおします．

In [5]:
# datasetの中身を全て統合する
all_data = []
for set_name in dataset.keys():
    all_data.extend(dataset[set_name])
# シャッフルし, 80%をtrain, 10%をvalidation, 10%をtestにする
import random

random.seed(42)
random.shuffle(all_data)
train_data = all_data[:int(0.8 * len(all_data))]
valid_data = all_data[int(0.8 * len(all_data)):int(0.9 * len(all_data))]
test_data = all_data[int(0.9 * len(all_data)):]
print(f"train: {len(train_data)}")
print(f"valid: {len(valid_data)}")
print(f"test: {len(test_data)}")

# 改めて，Datasets の形式にする
from datasets import Dataset

dataset_train = Dataset.from_list(train_data, features=dataset["train"].features)
dataset_valid = Dataset.from_list(valid_data, features=dataset["train"].features)
dataset_test = Dataset.from_list(test_data, features=dataset["train"].features)
dataset["train"] = dataset_train
dataset["validation"] = dataset_valid
dataset["test"] = dataset_test


train: 5893
valid: 737
test: 737


In [6]:
# 再度バランスを確認
for set_name in dataset.keys():
    category_counter = Counter([data["category"] for data in dataset[set_name]])
    print(f"{set_name}: {category_counter}")

train: Counter({6: 735, 0: 712, 7: 699, 8: 696, 2: 678, 5: 670, 1: 668, 3: 614, 4: 421})
validation: Counter({1: 102, 8: 98, 7: 94, 2: 88, 5: 87, 6: 83, 3: 72, 0: 71, 4: 42})
test: Counter({1: 100, 2: 98, 0: 87, 5: 85, 3: 84, 6: 82, 7: 77, 8: 76, 4: 48})


### データの前処理

データセットの内容を以下のように変更します:
- 入力として利用するために，記事のタイトルと本文を連結し，'text'というキーで保存します．
- 出力ラベルとして利用するために，カテゴリを'label'というキーで保存します．なお，カテゴリはすでに数値に変換されているため，そのまま利用します．

In [7]:
def preprocess(example):
    return {"text": example["title"] + "：" + example["content"],
            "label": example["category"]}
dataset = dataset.map(preprocess)

Map:   0%|          | 0/5893 [00:00<?, ? examples/s]

Map:   0%|          | 0/737 [00:00<?, ? examples/s]

Map:   0%|          | 0/737 [00:00<?, ? examples/s]

適切に変換されたか確かめてみましょう

In [8]:
pprint(dataset["train"][0])

{'category': 6,
 'content': '去る9月18日にボートの全日本選手権が行われ、女子シングルスカル決勝で若井江利が銀メダルを獲得した。  '
            '巷ではそれほど名の知れた存在ではないが、若井は知る人ぞ知る美人アスリート。競技で日焼けした小麦色の肌に、さわやかな笑顔を振りまき、ボート界では随一の美人選手として知られている。  '
            'スポーツタンクトップにサンバイザーをかける様は、これぞ健康美人という趣だ。  '
            '若井はボートで有名な岐阜県の加茂高校出身。高校在学中に総体ダブルスカルで優勝し、ジュニア選手権日本代表でも活躍。進学した早稲田大でも数々のタイトルを獲得し、2006年のアジア大会では日本代表として銀メダルを手にした。  '
            '現在は企業からスポンサードを受けながらフルタイムのボート選手として活動し、2010年アジア選手権では見事優勝を果たすなど、日本の女子ボート界を牽引するアスリートとして期待されている。  '
            '今回の全日本選手権での2位という結果については、自身のブログで「一番嫌いな色のメダルですが、現実をしっかり受け止めて、次へ進みたいと思います」とコメント。目標とするロンドン五輪での表彰台へ向け、全速力で水面を駆ける。  '
            '・若井江利フォトギャラリー',
 'date': '2011-09-27T08:30:00+0900',
 'label': 6,
 'text': '【Sports '
         'Watch】知る人ぞ知る美人アスリート、小麦肌の漕艇選手：去る9月18日にボートの全日本選手権が行われ、女子シングルスカル決勝で若井江利が銀メダルを獲得した。  '
         '巷ではそれほど名の知れた存在ではないが、若井は知る人ぞ知る美人アスリート。競技で日焼けした小麦色の肌に、さわやかな笑顔を振りまき、ボート界では随一の美人選手として知られている。  '
         'スポーツタンクトップにサンバイザーをかける様は、これぞ健康美人という趣だ。  '
         '若井はボートで有名な岐阜県の加茂高校出身。高校在学中に総体ダブルスカルで優勝し、ジュニア選

## モデルとトークナイザの準備

モデルには，東北大学が提供している[日本語BERTモデル](https://huggingface.co/cl-tohoku/bert-base-japanese-v3)を利用します．

### モデルとトークナイザの読み込み

ラベル数が9であることに注意して，`AutoModelForSequenceClassification`と`AutoTokenizer`でモデルとトークナイザを読み込みます．


In [9]:
label2id = {label: i for i, label in enumerate(dataset["train"].features["category"].names)}
id2label = {i: label for label, i in label2id.items()}

In [10]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "tohoku-nlp/bert-base-japanese-v3"
num_categories = 9

model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                           num_labels=num_categories,
                                                           id2label=id2label,
                                                           label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 前処理（トークナイズ）

学習をするために，データセットのテキストデータをトークナイズします．

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Parameter 'function'=<function tokenize_function at 0x7ff85fe9dc60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/5893 [00:00<?, ? examples/s]

Map:   0%|          | 0/737 [00:00<?, ? examples/s]

Map:   0%|          | 0/737 [00:00<?, ? examples/s]

In [12]:
pprint(list(tokenized_dataset["train"][0].keys()))

['url',
 'date',
 'title',
 'content',
 'category',
 'text',
 'label',
 'input_ids',
 'token_type_ids',
 'attention_mask']


長くなるので値の表示は割愛しますが，`input_id`，`attention_mask`，`token_type_ids`の3つのキーが追加されていることがわかります．

## 学習

### 評価関数

学習自体は`Trainer`によって損失関数が自動的に計算されて実行されますが，
どの程度の性能が出ているかは別途評価する必要があります．
そのための評価関数を用意しておきます．

In [13]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### 学習の設定

学習の設定（`TrainingArguments`）を準備します．

In [14]:
exp_name = "base"
output_dir = f"exp/{exp_name}/results"
logging_dir = f"exp/{exp_name}/logs"

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    num_train_epochs=10,            # 最大10エポックとする
    per_device_train_batch_size=8,  # バッチサイズ
    # auto_find_batch_size=True,    # バッチサイズを自動で見つける

    weight_decay=0.01,              # 重み減衰
    learning_rate=2e-5,             # 学習率
    warmup_steps=500,               # ウォームアップステップ数

    evaluation_strategy="epoch",    # 評価はエポックごとに行う
    metric_for_best_model="accuracy", # 最良のモデルの評価指標
    greater_is_better=True,           # 評価指標が大きいほど良い場合はTrue

    output_dir=output_dir,          # モデルの保存先
    save_strategy="epoch",          # モデルの保存はエポックごとに行う
    save_total_limit=3,             # 保存するモデルの数)

    logging_dir=logging_dir,        # ログの保存先
    logging_strategy="steps",       # ログの保存はエポックごとに行う
    logging_steps=100,              # 100ステップごとにログを出力する

    load_best_model_at_end=True,    # 最良のモデルを最後にロードする
)

/home/fujie/.conda/envs/py311/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### 早期終了用のコールバック

学習を途中で終了させるためのコールバックを用意します．
ここでは，3エポック以上，Accuracyが向上しない場合に学習を終了させるように設定します．

In [16]:
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3, 
    early_stopping_threshold=0.001)

### Trainerの作成

学習を行うためのTrainerを作成します．

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)

### 学習の実行

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.257300,0.196954,0.945726
2,0.125200,0.149820,0.957938
3,0.051500,0.199950,0.959294
4,0.016200,0.252548,0.960651
5,0.017300,0.193202,0.966079
6,0.000600,0.206794,0.967436
7,0.000200,0.203992,0.970149
8,0.000200,0.215309,0.971506
9,0.000100,0.209539,0.972863
10,0.000100,0.210000,0.972863


TrainOutput(global_step=7370, training_loss=0.10265099366778051, metrics={'train_runtime': 677.6127, 'train_samples_per_second': 86.967, 'train_steps_per_second': 10.876, 'total_flos': 1.550610899278848e+16, 'train_loss': 0.10265099366778051, 'epoch': 10.0})

### モデルの保存

In [19]:
model.save_pretrained(f"exp/{exp_name}/model")

### テストセットの評価

In [20]:
# dataset["test"]で評価
from transformers import pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, truncation=True)

results = classifier(dataset["test"]["text"])
# 結果のラベルを番号のリストに変換
predicted_labels = [label2id[result["label"]] for result in results]

# Accuracyの計算
metrics = metric.compute(predictions=predicted_labels, references=dataset["test"]["label"])

print(f"Test set accuracy: {metrics['accuracy']*100:.1f}%")

Test set accuracy: 97.3%


## ヘッド部のみの学習

前の例では，BERTの全体のパラメータをファインチューニングしました．
過学習する可能性が高まるため，ヘッド部のみをチューニングするようにします．

### モデルの再読み込み

まず，モデルの読み直しを行います．

In [21]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_categories)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### ベースモデルパラメータのフリーズ

次に，ベースモデルのパラメータをフリーズします．

In [22]:
# ベースモデルのフリーズ
for param in model.base_model.parameters():
    param.requires_grad = False

### Trainerの作成

モデルの保存先のみを変更します．

In [23]:
exp_name = "head"
training_args.output_dir = f"exp/{exp_name}/results"
training_args.logging_dir = f"exp/{exp_name}/logs"

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)

### 学習の実行

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,2.027200,1.954051,0.413840
2,1.752400,1.691338,0.624152
3,1.548100,1.506456,0.687924
4,1.392200,1.371511,0.715061
5,1.307200,1.278642,0.724559
6,1.216000,1.214706,0.724559
7,1.216100,1.168746,0.732700
8,1.158100,1.138648,0.735414
9,1.139400,1.121623,0.735414
10,1.165900,1.116052,0.735414


TrainOutput(global_step=7370, training_loss=1.4356679235740466, metrics={'train_runtime': 264.1855, 'train_samples_per_second': 223.063, 'train_steps_per_second': 27.897, 'total_flos': 1.550610899278848e+16, 'train_loss': 1.4356679235740466, 'epoch': 10.0})

### モデルの保存

In [26]:
model.save_pretrained(f"exp/{exp_name}/model")

### テストセットの評価

In [32]:
# dataset["test"]で評価
from transformers import pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, truncation=True)

results = classifier(dataset["test"]["text"])
# 結果のラベルを番号のリストに変換
label2id = classifier.model.config.label2id
predicted_labels = [label2id[result["label"]] for result in results]

# Accuracyの計算
metrics = metric.compute(predictions=predicted_labels, references=dataset["test"]["label"])

print(f"Test set accuracy: {metrics['accuracy']*100:.1f}%")

Test set accuracy: 73.8%
